In [1]:
import nltk
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize,sent_tokenize
from xlwt import Workbook
import os
import boto3
import pandas as pd

contents = []
bank_name= []
job_url= []
bigram_words= []
bigram_final= []
trigram_final= []

#Opens the CSV file from Amazon S3 and appends the values according to the column
client = boto3.client('s3',aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)
obj_job = client.get_object(Bucket='ads-assignment', Key='Job_Description.csv')
df = pd.read_csv(obj_job['Body'])
for i in df.columns:
    i
for ba in df['Bank Name']:
     bank_name.append(ba)
for ur in df['URL']:
     job_url.append(ur)
for de in df['Description']:
    contents.append(de)
    
#Opens a new workbook
wb = Workbook()
add = wb.add_sheet('Final_Count', cell_overwrite_ok=True)
row=1    
col=3
add.write(0,0, 'Bank Name')
add.write(0,1, 'URL')
add.write(0,2, 'Description')

#Opens the word file from Amazon S3 and appends it to contents1
contents1 = []

client = boto3.client('s3',aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)
obj = client.get_object(Bucket='ads-assignment', Key='Final_Words.csv')
df1 = pd.read_csv(obj['Body'])
for wor in df1['Final Words']:
    contents1.append(wor)
    
#Writes column names in the excel
for i in contents1:
    add.write(0,col,i)
    col = col +1
col_count=col

col=3
#Writes Bank names in the excel sheet at column 0
for bankname in bank_name[:]:
    bankname
    add.write(row,0,bankname)
    row = row +1    
    
row=1
#Writes URL in the excel sheet at column 1
for job_link in job_url[:]:
    job_link
    add.write(row,1,job_link)
    row=row+1
    
row=1
#Writes Job Desription in the excel sheet at column 2 and does the word count
row1=1
row2=1
for jobdata in contents[:]:
    word_tokens1 = word_tokenize(jobdata)
    words1 = [w for w in word_tokens1 if w.isalpha()]
    words_lowercase = [x.lower() for x in words1]

#Converting the description to Bigrams and Trigrams
    stop_words = set(stopwords.words('english'))
    nltk_tokens = nltk.word_tokenize(jobdata)
    words2 = [u for u in nltk_tokens if u.isalpha()]
    words2_lowercase = [y.lower() for y in words2]
    for z in words2_lowercase: 
        if z.lower() not in stop_words: 
            bigram_words.append(z)
    bigram = list(nltk.bigrams(bigram_words))
    trigram = list(nltk.trigrams(bigram_words))

#Formatting both the bigrams and trigrams
    for a in bigram:
        b=a[0]+" "+a[1]
        bigram_final.append(b)
    for c in trigram:
        d=c[0]+" "+c[1]+" "+c[2]
        trigram_final.append(d)

#Comparing the Unigram words in the description with the final fintech words
    for num in contents1:
        count = 0
        for num1 in words_lowercase:
            if num == num1:
                count = count+1
        add.write(row,2, jobdata)
        add.write(row,col,count)
            
        col = col+1
        if col == 105:
            col = 3
            row = row +1
            break
    
    col1=105
    
#Comparing the Bigram words in the description with the final bigram fintech words
    for num_bi in contents1[102:142]:
        count_bi = 0
        for num1_bi in bigram_final:
            if num_bi == num1_bi:
                count_bi = count_bi+1
        add.write(row1,col1,count_bi)
        col1 = col1+1
        if col1 == 145:
            col1 = 105
            row1 = row1 +1
    col2=145
#Comparing the Trigram words in the description with the final Trigram fintech words
    for num_tri in contents1[142:]:
        count_tri = 0
        for num1_tri in trigram_final:
            if num_tri == num1_tri:
                count_tri = count_tri+1
        add.write(row2,col2,count_tri)
        col2 = col2+1
        if col2 == col_count:
            col2 = 145
            row2 = row2 +1
    
    del words_lowercase[:]
    del bigram_final[:]
    del bigram[:]
    del bigram_words[:]
    del trigram[:]
    del trigram_final[:]

wb.save(os.getcwd()+'/Part1.csv')

s3 = boto3.resource('s3',aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)
bucket = s3.Bucket('ads-assignment')
s3.Object('ads-assignment', 'Part1.csv').put(Body=open(os.getcwd()+'/Part1.csv', 'rb'))


{'ResponseMetadata': {'RequestId': 'F50A496718EDE4C8',
  'HostId': 'mLiF3OkcOg/bhOE+r2KkGbQSIs4dL1Knw/R+AZIuY3DdJ5o3U16tzB04zE4M7Wz5HS7XtyRruaQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mLiF3OkcOg/bhOE+r2KkGbQSIs4dL1Knw/R+AZIuY3DdJ5o3U16tzB04zE4M7Wz5HS7XtyRruaQ=',
   'x-amz-request-id': 'F50A496718EDE4C8',
   'date': 'Wed, 27 Feb 2019 09:41:22 GMT',
   'etag': '"011a7d905f17a210bb7ca2dcbfdfdd1f"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"011a7d905f17a210bb7ca2dcbfdfdd1f"'}